# ADC multi 3rd multi1
From an analog input, produce a digital output. We explore multiple architectures.

# Preamble

In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib widget
%matplotlib inline

In [2]:
import bqplot as bq
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.max_open_warning'] = 0
import numpy as np
from sidecar import Sidecar
import time

Fetch our tools:

In [3]:
from lib.nnn import Network, Layer, IdentityLayer, AffineLayer, MapLayer
from lib.nnbench import NNBench, NetMaker
from lib.nnvis import NNVis, ADCResponsePlot, NetResponsePlot

In [4]:
def plot_ADC(fun, **kwargs):
    return ADCResponsePlot(**kwargs)(fun).fig

## Reference implementation
Here's what we want to accomplish, but by network means:

### 3-bit linear binary output

In [5]:
def adc(input):
    m = max(0, min(7, int(8*input)))
    return np.array([(m>>2)&1, (m>>1)&1, m&1]) * 2 - 1

vadc = lambda v: np.array([adc(p) for p in v])
#plot_ADC(vadc)

### 3-bit linear Gray coded output

In [6]:
def gradc(input):
    m = max(0, min(7, int(8*input)))
    g = m ^ (m >> 1)
    return np.array([(g>>2)&1, (g>>1)&1, g&1]) * 2 - 1

vgradc = lambda v: np.array([gradc(p) for p in v])
#plot_ADC(vgradc)

___

# Network
We create a multiplicity of the same ADC net

In [7]:
mnet = Network()
mnet.extend(AffineLayer(1,8, multiples=5))
mnet.extend(MapLayer(np.tanh, lambda d: 1.0 - np.tanh(d) ** 2))
mnet.extend(AffineLayer(8,8, multiples=5))
mnet.extend(MapLayer(np.tanh, lambda d: 1.0 - np.tanh(d) ** 2))
mnet.extend(AffineLayer(8,3, multiples=5))
mnet.extend(MapLayer(np.tanh, lambda d: 1.0 - np.tanh(d) ** 2))
mnet.extend(AffineLayer(3,3, multiples=5))
mnet.extend(MapLayer(np.tanh, lambda d: 1.0 - np.tanh(d) ** 2))

In [9]:
mnet.eta = 0.01

# Graphs to the right

In [10]:
grid = widgets.GridspecLayout(3, 2, height='680px',
                      grid_gap='10px',
                      justify_content='center',
                      align_items='top')

#nrps = np.array([NetResponsePlot(net, height='220px', margin=30) \
#            for i in range(grid.n_rows * grid.n_columns)]).reshape(grid.n_rows, grid.n_columns)
nrps = np.array([NetResponsePlot(net, height='220px', margin=30, title='title') for net in mnet])

for i, nrp in enumerate(nrps):
    column = i % grid.n_columns
    row = i // grid.n_columns
    grid[row, column] = nrp.fig

batch_w = widgets.FloatText(value=-1.0, description='Batch:', max_width=6, disabled=True)
eta_w = widgets.FloatLogSlider(
    value=starting_eta,
    base=10,
    min=-4,
    max=0, # min exponent of base
    step=0.1, # exponent step
    description='eta'
)

def on_eta_change(change):
    eta = change['new']
    mnet.eta = eta

eta_w.observe(on_eta_change, names='value')

grid[-1,-1] = widgets.VBox((batch_w, eta_w))
        
with Sidecar(title='grid') as gside:
    display(grid)

TypeError: 'Network' object is not iterable

In [ ]:
_ = [nrp() for nrp in nrps]

# Training
We will train our candidates in parallel, on the same training data, and watch their evolution.

## Training data

In [ ]:
x = np.arange(0, 1, 1.0/(8*8)).reshape(-1,1) # 8 points in each output region
training_batch_cluster = [(x, vadc(x))]

In [ ]:
#training_batch_cluster

## Training loop

In [ ]:
batch = 0

In [ ]:
for i in range(10):
    for net in nets:
        net.learn(training_batch_cluster)
    _ = [nrp() for nrp in nrps]
    batch += 1
    batch_w.value = batch
    time.sleep(0.5)

In [ ]:
for i in range(100):
    for net in nets:
        net.learn(training_batch_cluster)
    batch += 1
    if i % 10 == 0:
        _ = [nrp() for nrp in nrps]
        batch_w.value = batch

In [ ]:
for i in range(100_000):
    for net in nets:
        net.learn(training_batch_cluster)
    batch += 1
    if i % 100 == 0:
        _ = [nrp() for nrp in nrps]
        batch_w.value = batch

In [ ]:
for i in range(100_000):
    for net in nets:
        net.learn(training_batch_cluster)
    batch += 1
    if i % 500 == 0:
        _ = [nrp() for nrp in nrps]
        batch_w.value = batch

In [ ]:
assert False, "stop here"

In [ ]:
for i in range(100_000):
    for j in range(min(int(2**(i/4)), 500)):
        for net in nets:
            net.learn(training_batch_cluster)
        batch += 1
    _ = [nrp() for nrp in nrps]
    batch_w.value = batch

In [ ]:
nrps[0]()

In [ ]:
nrps[1]()

In [ ]:
nrps[3].net.layers

In [ ]:
nets

In [ ]:
nrps

In [ ]:
[hash(nrp.fig) for nrp in nrps]

In [ ]:
nrps[3].net(np.array([0.5]))